<a href="https://colab.research.google.com/github/RamadhikaS/STKI/blob/main/KlasifikasiDokumen_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# --- BAGIAN 1: PERSIAPAN DATA ---
# Data training dan testing dari soal
data_training = {
    'D1': 'Sepakbola PSIS tahun ini semakin indah.',
    'D2': 'Presiden Indonesia menaikan harga BBM.',
    'D3': 'Partai politik Indonesia berburu suara.',
    'D4': 'Timnas Indonesia gagal juara AFC.'
}
label_training = {
    'D1': 'Olahraga',
    'D2': 'Politik',
    'D3': 'Politik',
    'D4': 'Olahraga'
}
data_testing = {
    'D5': 'Presiden menaikan gaji Timnas Indonesia'
}
k = 3

# Gabungkan semua dokumen untuk proses vektorisasi
semua_dokumen = list(data_training.values()) + list(data_testing.values())
nama_dokumen = list(data_training.keys()) + list(data_testing.keys())


# --- BAGIAN 2: VEKTORISASI MENGGUNAKAN TF-IDF ---
# TF-IDF adalah cara populer untuk mengubah teks menjadi vektor numerik
# Ini memberi bobot lebih tinggi pada kata yang sering muncul di satu dokumen
# tetapi jarang muncul di dokumen lain (dianggap lebih penting).

# Membuat model TfidfVectorizer
vectorizer = TfidfVectorizer()

# Proses training dan transformasi data teks menjadi matriks TF-IDF
tfidf_matrix = vectorizer.fit_transform(semua_dokumen)

# Mengubah hasil menjadi DataFrame agar mudah dibaca
df_tfidf = pd.DataFrame(
    tfidf_matrix.toarray(),
    index=nama_dokumen,
    columns=vectorizer.get_feature_names_out()
)

print("===== Matriks TF-IDF (Vektor Dokumen) =====")
print(df_tfidf)
print("\n" + "="*50 + "\n")


# --- BAGIAN 3: MENGHITUNG KEMIRIPAN & MENEMUKAN TETANGGA ---

# Ambil vektor TF-IDF untuk dokumen training dan testing
vektor_training = tfidf_matrix[:-1] # Ambil semua kecuali baris terakhir (D5)
vektor_testing = tfidf_matrix[-1]  # Ambil hanya baris terakhir (D5)

# Hitung Cosine Similarity antara D5 dengan semua dokumen training (D1-D4)
similarity_scores = cosine_similarity(vektor_testing, vektor_training)

# Buat DataFrame untuk menampilkan hasil kemiripan
df_similarity = pd.DataFrame({
    'Dokumen': list(data_training.keys()),
    'Class': list(label_training.values()),
    'Similarity': similarity_scores[0] # Ambil baris pertama karena hanya ada 1 data testing
})

# Urutkan berdasarkan similarity tertinggi
df_similarity = df_similarity.sort_values(by='Similarity', ascending=False)

print("===== Kemiripan D5 dengan Dokumen Training =====")
print(df_similarity)
print("\n" + "="*50 + "\n")

# Ambil 'k' tetangga terdekat
nearest_neighbors = df_similarity.head(k)

print(f"===== {k} Tetangga Terdekat (k-Nearest Neighbors) =====")
print(nearest_neighbors)
print("\n" + "="*50 + "\n")


# --- BAGIAN 4: VOTING DAN KESIMPULAN ---

# Lakukan voting berdasarkan kelas dari tetangga terdekat
voting_result = nearest_neighbors['Class'].mode()

# Ambil hasil mayoritas (mode bisa menghasilkan lebih dari 1 jika ada seri)
predicted_class = voting_result[0]

print("===== Hasil Klasifikasi untuk D5 =====")
print(f"Kelas dari {k} tetangga terdekat adalah: {list(nearest_neighbors['Class'])}")
print(f"Hasil voting (kelas mayoritas) adalah: '{predicted_class}'")
print(f"\nKesimpulan: Dokumen D5 diklasifikasikan sebagai kelas **{predicted_class}**.")

===== Matriks TF-IDF (Vektor Dokumen) =====
         afc       bbm  berburu     gagal      gaji     harga     indah  \
D1  0.000000  0.000000  0.00000  0.000000  0.000000  0.000000  0.408248   
D2  0.000000  0.525644  0.00000  0.000000  0.000000  0.525644  0.000000   
D3  0.000000  0.000000  0.48127  0.000000  0.000000  0.000000  0.000000   
D4  0.501992  0.000000  0.00000  0.501992  0.000000  0.000000  0.000000   
D5  0.000000  0.000000  0.00000  0.000000  0.552989  0.000000  0.000000   

    indonesia       ini     juara  menaikan   partai  politik  presiden  \
D1   0.000000  0.408248  0.000000  0.000000  0.00000  0.00000  0.000000   
D2   0.296139  0.000000  0.000000  0.424086  0.00000  0.00000  0.424086   
D3   0.271139  0.000000  0.000000  0.000000  0.48127  0.48127  0.000000   
D4   0.282814  0.000000  0.501992  0.000000  0.00000  0.00000  0.000000   
D5   0.311544  0.000000  0.000000  0.446148  0.00000  0.00000  0.446148   

        psis   semakin  sepakbola    suara     tahun  